In [22]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import random

In [23]:
# CUDA
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE.type

'cuda'

In [24]:
# Fix random seed
seed = 42
torch.manual_seed(seed)
if DEVICE.type == 'cuda':
	torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Pre-processing input data

In [25]:
def bits_to_MiB(row):
	# verify if has string ' MiB'
	if 'MiB' in str(row):
		row = row.replace(' MiB', '')
		row = float(row)
	else:
		row = float(row) / np.power(2, 20)
	return row


def MHz_to_GHz(row):
	# verify if has string ' GHz'
	if 'GHz' in str(row):
		row = row.replace(' GHz', '')
		# convert to float
		row = float(row)
	else:
		row = row.replace(' MHz', '')
		row = float(row) / 1000
	return row

In [26]:
results_df = pd.read_csv('../results/execution_time.csv')
results_savio_df = pd.read_csv('../results_savio/execution_time.csv')
results_df = pd.concat([results_df, results_savio_df], ignore_index=True)
# preprocessing
results_df['total_cpu_usage'] = results_df['total_cpu_usage'].str.replace('%', '').astype(float) / 100
results_df['max_ram_usage'] = results_df['max_ram_usage'] / 1024
results_df['l2_cache_size'] = results_df['l2_cache_size'].apply(bits_to_MiB)
results_df['l3_cache_size'] = results_df['l3_cache_size'].apply(bits_to_MiB)
results_df['ghz_actual_friendly'] = results_df['hz_actual_friendly'].apply(MHz_to_GHz)
results_df['ghz_advertised_friendly'] = results_df['hz_advertised_friendly'].str.replace('GHz', '').astype(float)
results_df = results_df.drop(columns=['hz_actual_friendly', 'hz_advertised_friendly', 'arch', 'vendor_id_raw'])

In [27]:
# remove one computer for testing
g_train = results_df[results_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['benchmark','brand_raw'])
g_test = results_df[results_df['brand_raw'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['benchmark','brand_raw'])

In [28]:
mm_df = results_df[results_df['benchmark']=='MATRIX_MULT'].drop(columns=['benchmark'])
# remove one computer for testing
mm_train = mm_df[mm_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw'])
mm_test = mm_df[mm_df['brand_raw'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw'])

In [29]:
st_df = results_df[results_df['benchmark']!='MATRIX_MULT'].drop(columns=['benchmark','count'])
# remove one computer for testing
st_train = st_df[st_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw'])
st_test = st_df[st_df['brand_raw'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw'])

In [30]:
target = 'total_time'
features = mm_test.columns.copy().drop(target)
features_st = features.copy().drop(['count'])

In [31]:
# general data
## split data
X_g_train = g_train[features]
y_g_train = g_train[target]

X_g_test = g_test[features]
y_g_test = g_test[target]

## normalize data
scaler = StandardScaler()
X_g_train = scaler.fit_transform(X_g_train)
X_g_test = scaler.transform(X_g_test)

## convert to tensor
X_g_train = torch.tensor(X_g_train, dtype=torch.float32).unsqueeze(1)
X_g_test = torch.tensor(X_g_test, dtype=torch.float32).unsqueeze(1)
y_g_train = torch.tensor(y_g_train.values, dtype=torch.float32).view(-1, 1)
y_g_test = torch.tensor(y_g_test.values, dtype=torch.float32).view(-1, 1)

In [32]:
# single thread data
## split data
X_st_train = st_train[features_st]
y_st_train = st_train[target]

X_st_test = st_test[features_st]
y_st_test = st_test[target]

## normalize data
scaler = StandardScaler()
X_st_train = scaler.fit_transform(X_st_train)
X_st_test = scaler.transform(X_st_test)

## convert to tensor
X_st_train = torch.tensor(X_st_train, dtype=torch.float32).unsqueeze(1)
X_st_test = torch.tensor(X_st_test, dtype=torch.float32).unsqueeze(1)
y_st_train = torch.tensor(y_st_train.values, dtype=torch.float32).view(-1, 1)
y_st_test = torch.tensor(y_st_test.values, dtype=torch.float32).view(-1, 1)

In [33]:
# multi thread data
## split data
X_mm_train = mm_train[features]
y_mm_train = mm_train[target]

X_mm_test = mm_test[features]
y_mm_test = mm_test[target]

## normalize data
scaler = StandardScaler()
X_mm_train = scaler.fit_transform(X_mm_train)
X_mm_test = scaler.transform(X_mm_test)

## convert to tensor
X_mm_train = torch.tensor(X_mm_train, dtype=torch.float32).unsqueeze(1)
X_mm_test = torch.tensor(X_mm_test, dtype=torch.float32).unsqueeze(1)
y_mm_train = torch.tensor(y_mm_train.values, dtype=torch.float32).view(-1, 1)
y_mm_test = torch.tensor(y_mm_test.values, dtype=torch.float32).view(-1, 1)

In [34]:
if DEVICE.type == 'cuda':
	# move to DEVICE
	X_g_train = X_g_train.to(DEVICE)
	y_g_train = y_g_train.to(DEVICE)
	X_g_test = X_g_test.to(DEVICE)
	y_g_test = y_g_test.to(DEVICE)

	X_st_train = X_st_train.to(DEVICE)
	y_st_train = y_st_train.to(DEVICE)
	X_st_test = X_st_test.to(DEVICE)
	y_st_test = y_st_test.to(DEVICE)

	X_mm_train = X_mm_train.to(DEVICE)
	y_mm_train = y_mm_train.to(DEVICE)
	X_mm_test = X_mm_test.to(DEVICE)
	y_mm_test = y_mm_test.to(DEVICE)

# Model

In [35]:
class TransformerModel(nn.Module):
	def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim, dropout=0.1):
		super(TransformerModel, self).__init__()
		# layers
		self.embedding = nn.Linear(input_dim, model_dim)
		encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, batch_first=True)
		self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
		self.fc = nn.Linear(model_dim, output_dim)
		self.dropout = nn.Dropout(dropout)
	
	def forward(self, x):
		x = self.embedding(x)
		x = self.dropout(x)
		x = self.transformer(x)
		x = self.fc(x.mean(dim=1))
		return x

In [36]:
def objective(trial: optuna.Trial, X_train, y_train, X_test, y_test, input_dim, output_dim):
	# Definimos los hiperparámetros a buscar
	num_heads = trial.suggest_int('num_heads', 1, 8)
	model_dim = trial.suggest_int('model_dim', num_heads * 4, num_heads * 64, step=num_heads)
	num_layers = trial.suggest_int('num_layers', 1, 6)
	dropout = trial.suggest_float('dropout', 0.1, 0.5)
	learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
	weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
	num_epochs = trial.suggest_int('num_epochs', 10, 100)

	# model initialization 
	model = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
	if DEVICE.type == 'cuda':
		model = model.to(DEVICE)
	criterion = nn.MSELoss()
	optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
	# training
	model.train()
	for epoch in range(num_epochs):
		optimizer.zero_grad()
		output = model(X_train)
		loss = criterion(output, y_train)
		loss.backward()
		optimizer.step()
	# evaluation
	model.eval()
	with torch.no_grad():
		predictions = model(X_test)
		val_loss = criterion(predictions, y_test)

		# trial.report(val_loss.item(), epoch+1)
		# if trial.should_prune():
		# 	raise optuna.TrialPruned()
	print(f"Trial: {trial.number} - Loss: {loss.item()} - Val Loss: {val_loss.item()}")
	return val_loss.item()

# Hyperparameters Optimization

In [37]:
n_trials = 100

## General

In [38]:
# configuration optuna
study_g = optuna.create_study(direction='minimize')
study_g.optimize(lambda trial: objective(trial, X_g_train, y_g_train, X_g_test, y_g_test, len(features), len(target)), n_trials=n_trials)

[I 2024-06-20 05:14:35,859] A new study created in memory with name: no-name-8e475bdb-b813-46eb-85f3-ee8d5b669858
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([180, 1])) that is different to the input size (torch.Size([180, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([20, 1])) that is different to the input size (torch.Size([20, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2024-06-20 05:14:36,320] Trial 0 finished with value: 870.7213745117188 and parameters: {'num_heads': 2, 'model_dim':

Trial: 0 - Loss: 386.89605712890625 - Val Loss: 870.7213745117188


[I 2024-06-20 05:14:36,693] Trial 1 finished with value: 152.57757568359375 and parameters: {'num_heads': 4, 'model_dim': 112, 'num_layers': 4, 'dropout': 0.1124448114212703, 'learning_rate': 0.0036271528609092764, 'weight_decay': 7.074711477758549e-05, 'num_epochs': 52}. Best is trial 1 with value: 152.57757568359375.
[I 2024-06-20 05:14:36,859] Trial 2 finished with value: 358.3475341796875 and parameters: {'num_heads': 6, 'model_dim': 42, 'num_layers': 1, 'dropout': 0.13064577969827562, 'learning_rate': 0.008237486522403943, 'weight_decay': 0.0005536388059568095, 'num_epochs': 70}. Best is trial 1 with value: 152.57757568359375.


Trial: 1 - Loss: 47.18220520019531 - Val Loss: 152.57757568359375
Trial: 2 - Loss: 12.7821044921875 - Val Loss: 358.3475341796875


[I 2024-06-20 05:14:37,216] Trial 3 finished with value: 600.9075927734375 and parameters: {'num_heads': 6, 'model_dim': 54, 'num_layers': 3, 'dropout': 0.3503944347509199, 'learning_rate': 0.001086155949063208, 'weight_decay': 0.004744834918674038, 'num_epochs': 66}. Best is trial 1 with value: 152.57757568359375.


...

Trial: 97 - Loss: 180.19195556640625 - Val Loss: 501.974609375


[I 2024-06-20 05:15:43,371] Trial 98 finished with value: 125.54471588134766 and parameters: {'num_heads': 8, 'model_dim': 424, 'num_layers': 5, 'dropout': 0.3108038702202631, 'learning_rate': 0.0027864372008787244, 'weight_decay': 0.002041763338677023, 'num_epochs': 22}. Best is trial 74 with value: 78.46845245361328.


Trial: 98 - Loss: 50.62577819824219 - Val Loss: 125.54471588134766


[I 2024-06-20 05:15:43,758] Trial 99 finished with value: 136.9536590576172 and parameters: {'num_heads': 8, 'model_dim': 408, 'num_layers': 5, 'dropout': 0.3672526079848649, 'learning_rate': 0.005570518825588713, 'weight_decay': 0.003152519827176785, 'num_epochs': 18}. Best is trial 74 with value: 78.46845245361328.


Trial: 99 - Loss: 50.76043701171875 - Val Loss: 136.9536590576172


In [39]:
# Results
print(f'Número de pruebas: {len(study_g.trials)}')
trial = study_g.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida: {trial.value}')

Número de pruebas: 100
Mejor prueba: 74
Mejores parametros: {'num_heads': 8, 'model_dim': 496, 'num_layers': 6, 'dropout': 0.4051282113428471, 'learning_rate': 0.004309566957857929, 'weight_decay': 0.0005126147306951872, 'num_epochs': 15}
Mejor valor de pérdida: 78.46845245361328


## Single Thread

In [40]:
# configuration optuna
study_st = optuna.create_study(direction='minimize')
study_st.optimize(lambda trial: objective(trial, X_st_train, y_st_train, X_st_test, y_st_test, len(features_st), len(target)), n_trials=n_trials)

[I 2024-06-20 05:15:43,789] A new study created in memory with name: no-name-9dc7cf60-4bd9-4f75-a316-8a9fbde6de7c
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([135, 1])) that is different to the input size (torch.Size([135, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([15, 1])) that is different to the input size (torch.Size([15, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2024-06-20 05:15:44,019] Trial 0 finished with value: 553.1808471679688 and parameters: {'num_heads': 1, 'model_dim':

Trial: 0 - Loss: 348.5758361816406 - Val Loss: 553.1808471679688


[I 2024-06-20 05:15:44,651] Trial 1 finished with value: 585.1207885742188 and parameters: {'num_heads': 3, 'model_dim': 66, 'num_layers': 5, 'dropout': 0.13740167606562004, 'learning_rate': 6.610409328626004e-05, 'weight_decay': 0.0004543254939001672, 'num_epochs': 84}. Best is trial 0 with value: 553.1808471679688.
[I 2024-06-20 05:15:44,717] Trial 2 finished with value: 87.58348083496094 and parameters: {'num_heads': 5, 'model_dim': 115, 'num_layers': 1, 'dropout': 0.4657496616949821, 'learning_rate': 0.005281045421205645, 'weight_decay': 0.00032554083501477204, 'num_epochs': 24}. Best is trial 2 with value: 87.58348083496094.


Trial: 1 - Loss: 371.4375305175781 - Val Loss: 585.1207885742188
Trial: 2 - Loss: 39.95188522338867 - Val Loss: 87.58348083496094


[I 2024-06-20 05:15:45,286] Trial 3 finished with value: 446.05084228515625 and parameters: {'num_heads': 2, 'model_dim': 46, 'num_layers': 6, 'dropout': 0.39156839668455323, 'learning_rate': 0.0011196332640909215, 'weight_decay': 0.0010555210109952533, 'num_epochs': 60}. Best is trial 2 with value: 87.58348083496094.
[I 2024-06-20 05:15:45,359] Trial 4 finished with value: 26.357051849365234 and parameters: {'num_heads': 7, 'model_dim': 133, 'num_layers': 2, 'dropout': 0.1780524768826179, 'learning_rate': 0.00968967984581368, 'weight_decay': 3.710073967520182e-05, 'num_epochs': 16}. Best is trial 4 with value: 26.357051849365234.


...

Trial: 94 - Loss: 23.919204711914062 - Val Loss: 26.174816131591797
Trial: 95 - Loss: 20.95515251159668 - Val Loss: 18.310789108276367
Trial: 96 - Loss: 37.481632232666016 - Val Loss: 22.01729965209961


[I 2024-06-20 05:16:05,973] Trial 97 finished with value: 506.68499755859375 and parameters: {'num_heads': 7, 'model_dim': 406, 'num_layers': 1, 'dropout': 0.3044650466291039, 'learning_rate': 0.0002183287025599589, 'weight_decay': 0.0020186047922250965, 'num_epochs': 10}. Best is trial 60 with value: 18.217418670654297.
[I 2024-06-20 05:16:06,077] Trial 98 finished with value: 38.292816162109375 and parameters: {'num_heads': 7, 'model_dim': 336, 'num_layers': 1, 'dropout': 0.34463495945821787, 'learning_rate': 0.004672473212254047, 'weight_decay': 0.0009508262969350424, 'num_epochs': 14}. Best is trial 60 with value: 18.217418670654297.


Trial: 97 - Loss: 316.8341979980469 - Val Loss: 506.68499755859375
Trial: 98 - Loss: 20.514841079711914 - Val Loss: 38.292816162109375


[I 2024-06-20 05:16:06,215] Trial 99 finished with value: 28.21881866455078 and parameters: {'num_heads': 7, 'model_dim': 399, 'num_layers': 1, 'dropout': 0.3272770550863353, 'learning_rate': 0.008273060955363027, 'weight_decay': 0.0024668450979883016, 'num_epochs': 26}. Best is trial 60 with value: 18.217418670654297.


Trial: 99 - Loss: 17.451608657836914 - Val Loss: 28.21881866455078


In [41]:
# Results
print(f'Número de pruebas: {len(study_st.trials)}')
trial = study_st.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Número de pruebas: 100
Mejor prueba: 60
Mejores parametros: {'num_heads': 5, 'model_dim': 305, 'num_layers': 4, 'dropout': 0.3944508870189298, 'learning_rate': 0.0052776377703212364, 'weight_decay': 0.0007789453603024966, 'num_epochs': 19}
Mejor valor de pérdida en validación: 18.217418670654297


## Multi Thread

In [51]:
# configuration optuna
study_mm = optuna.create_study(direction='minimize')
study_mm.optimize(lambda trial: objective(trial, X_mm_train, y_mm_train, X_mm_test, y_mm_test, len(features), len(target)), n_trials=n_trials)

[I 2024-06-20 05:18:59,957] A new study created in memory with name: no-name-7b56daef-3104-41c0-bbb0-11a969de7cbc
[I 2024-06-20 05:19:00,174] Trial 0 finished with value: 1367.906982421875 and parameters: {'num_heads': 6, 'model_dim': 342, 'num_layers': 3, 'dropout': 0.35729459721182055, 'learning_rate': 4.41927159482419e-05, 'weight_decay': 0.005492516187005042, 'num_epochs': 34}. Best is trial 0 with value: 1367.906982421875.
[I 2024-06-20 05:19:00,310] Trial 1 finished with value: 1391.062744140625 and parameters: {'num_heads': 2, 'model_dim': 64, 'num_layers': 2, 'dropout': 0.17007905297777648, 'learning_rate': 0.0008346083363926891, 'weight_decay': 0.0005018227075829743, 'num_epochs': 35}. Best is trial 0 with value: 1367.906982421875.


Trial: 0 - Loss: 247.82350158691406 - Val Loss: 1367.906982421875
Trial: 1 - Loss: 257.7347106933594 - Val Loss: 1391.062744140625


[I 2024-06-20 05:19:00,609] Trial 2 finished with value: 1587.4559326171875 and parameters: {'num_heads': 1, 'model_dim': 15, 'num_layers': 2, 'dropout': 0.35126842549103066, 'learning_rate': 5.2298128469062746e-05, 'weight_decay': 0.00024029470137962227, 'num_epochs': 80}. Best is trial 0 with value: 1367.906982421875.
[I 2024-06-20 05:19:00,780] Trial 3 finished with value: 1457.5709228515625 and parameters: {'num_heads': 6, 'model_dim': 48, 'num_layers': 1, 'dropout': 0.17460812334645084, 'learning_rate': 0.0003653766634842455, 'weight_decay': 0.0024674739592383654, 'num_epochs': 71}. Best is trial 0 with value: 1367.906982421875.


Trial: 2 - Loss: 321.6813049316406 - Val Loss: 1587.4559326171875
Trial: 3 - Loss: 278.43524169921875 - Val Loss: 1457.5709228515625


[I 2024-06-20 05:19:01,052] Trial 4 finished with value: 1438.416259765625 and parameters: {'num_heads': 3, 'model_dim': 129, 'num_layers': 5, 'dropout': 0.4425061033003763, 'learning_rate': 0.00011597522524470304, 'weight_decay': 0.0003389139749388, 'num_epochs': 26}. Best is trial 0 with value: 1367.906982421875.
[I 2024-06-20 05:19:01,238] Trial 5 finished with value: 1559.0189208984375 and parameters: {'num_heads': 1, 'model_dim': 55, 'num_layers': 1, 'dropout': 0.1647183033809928, 'learning_rate': 4.137807220691872e-05, 'weight_decay': 3.577739784888944e-05, 'num_epochs': 69}. Best is trial 0 with value: 1367.906982421875.


...

Trial: 96 - Loss: 124.42733001708984 - Val Loss: 475.50726318359375
Trial: 97 - Loss: 21.338220596313477 - Val Loss: 910.853515625


[I 2024-06-20 05:19:30,256] Trial 98 finished with value: 612.6205444335938 and parameters: {'num_heads': 8, 'model_dim': 456, 'num_layers': 6, 'dropout': 0.40998239472432446, 'learning_rate': 0.0034752413096067873, 'weight_decay': 0.0005351798891154827, 'num_epochs': 94}. Best is trial 85 with value: 410.5378112792969.


Trial: 98 - Loss: 113.30770874023438 - Val Loss: 612.6205444335938


[I 2024-06-20 05:19:30,468] Trial 99 finished with value: 503.0196838378906 and parameters: {'num_heads': 8, 'model_dim': 200, 'num_layers': 6, 'dropout': 0.23239493318264295, 'learning_rate': 0.00858599968155528, 'weight_decay': 0.0041674387650437366, 'num_epochs': 17}. Best is trial 85 with value: 410.5378112792969.


Trial: 99 - Loss: 123.97649383544922 - Val Loss: 503.0196838378906


In [52]:
# Results
print(f'Trials quantity: {len(study_mm.trials)}')
trial = study_mm.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Trials quantity: 100
Mejor prueba: 85
Mejores parametros: {'num_heads': 7, 'model_dim': 329, 'num_layers': 6, 'dropout': 0.25966838018766203, 'learning_rate': 0.00961115924403532, 'weight_decay': 0.0012671115668168938, 'num_epochs': 10}
Mejor valor de pérdida en validación: 410.5378112792969


# Training

In [44]:
output_dim = 1

## General

In [45]:
input_dim = len(features)
# hyperparameters
num_heads = study_g.best_trial.params['num_heads']
model_dim = study_g.best_trial.params['model_dim']
num_layers = study_g.best_trial.params['num_layers']
dropout = study_g.best_trial.params['dropout']
lr = study_g.best_trial.params['learning_rate']
wd = study_g.best_trial.params['weight_decay']
num_epochs = study_g.best_trial.params['num_epochs']

study_g.best_trial.params

{'num_heads': 8,
 'model_dim': 496,
 'num_layers': 6,
 'dropout': 0.4051282113428471,
 'learning_rate': 0.004309566957857929,
 'weight_decay': 0.0005126147306951872,
 'num_epochs': 15}

In [46]:
# general model initialization
model_g = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_g = model_g.to(DEVICE)
criterion_g = nn.MSELoss()
optimizer_g = optim.AdamW(model_g.parameters(), lr=lr, weight_decay=wd)

model_g.train()

for epoch in range(num_epochs):
	optimizer_g.zero_grad()
	output = model_g(X_g_train)
	loss = criterion_g(output, y_g_train)
	loss.backward()
	optimizer_g.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_g.eval()
		with torch.no_grad():
			val_predictions = model_g(X_g_test)
			val_loss = criterion_g(val_predictions, y_g_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_g.train()

Epoch 10/15, Loss: 52.74978256225586, Val Loss: 105.2210693359375
Epoch 15/15, Loss: 49.432525634765625, Val Loss: 166.17543029785156


## Single Thread

In [47]:
input_dim = len(features_st)
# hyperparameters
num_heads = study_st.best_trial.params['num_heads']
model_dim = study_st.best_trial.params['model_dim']
num_layers = study_st.best_trial.params['num_layers']
dropout = study_st.best_trial.params['dropout']
lr = study_st.best_trial.params['learning_rate']
wd = study_st.best_trial.params['weight_decay']
num_epochs = study_st.best_trial.params['num_epochs']
study_st.best_trial.params

{'num_heads': 5,
 'model_dim': 305,
 'num_layers': 4,
 'dropout': 0.3944508870189298,
 'learning_rate': 0.0052776377703212364,
 'weight_decay': 0.0007789453603024966,
 'num_epochs': 19}

In [48]:
# single thread model initialization
model_st = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_st = model_st.to(DEVICE)
criterion_st = nn.MSELoss()
optimizer_st = optim.AdamW(model_st.parameters(), lr=lr, weight_decay=wd)

model_st.train()

for epoch in range(num_epochs):
	optimizer_st.zero_grad()
	output = model_st(X_st_train)
	loss = criterion_st(output, y_st_train)
	loss.backward()
	optimizer_st.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_st.eval()
		with torch.no_grad():
			val_predictions = model_st(X_st_test)
			val_loss = criterion_st(val_predictions, y_st_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_st.train()

Epoch 10/19, Loss: 17.625991821289062, Val Loss: 30.301942825317383
Epoch 19/19, Loss: 18.42536163330078, Val Loss: 35.388484954833984


## Multi Thread

In [49]:
input_dim = len(features)
# hyperparameters
num_heads = study_mm.best_trial.params['num_heads']
model_dim = study_mm.best_trial.params['model_dim']
num_layers = study_mm.best_trial.params['num_layers']
dropout = study_mm.best_trial.params['dropout']
lr = study_mm.best_trial.params['learning_rate']
wd = study_mm.best_trial.params['weight_decay']
num_epochs = study_mm.best_trial.params['num_epochs']

study_mm.best_trial.params

{'num_heads': 8,
 'model_dim': 472,
 'num_layers': 3,
 'dropout': 0.3497167919307952,
 'learning_rate': 0.0027713172701841183,
 'weight_decay': 0.005883185183789469,
 'num_epochs': 41}

In [50]:
# multi thread model initialization
model_mm = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_mm = model_mm.to(DEVICE)
criterion_mm = nn.MSELoss()
optimizer_mm = optim.AdamW(model_mm.parameters(), lr=lr, weight_decay=wd)

model_mm.train()

for epoch in range(num_epochs):
	optimizer_mm.zero_grad()
	output = model_mm(X_mm_train)
	loss = criterion_mm(output, y_mm_train)
	loss.backward()
	optimizer_mm.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_mm.eval()
		with torch.no_grad():
			val_predictions = model_mm(X_mm_test)
			val_loss = criterion_mm(val_predictions, y_mm_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_mm.train()

Epoch 10/41, Loss: 53.83079528808594, Val Loss: 427.0614929199219
Epoch 20/41, Loss: 35.6347541809082, Val Loss: 769.7138671875
Epoch 30/41, Loss: 35.99632263183594, Val Loss: 926.7322387695312
Epoch 40/41, Loss: 34.37905502319336, Val Loss: 1228.2109375
Epoch 41/41, Loss: 34.84366226196289, Val Loss: 1210.6322021484375


# Conclusion
Queda trabajo que hacer en la red, además de conseguir más datos para un entrenamiento más robusto. Queda por ahora descartado el uso de solo un modelo para multi-threading y single-threading, ya que el modelo tiene más del triple de *loss*.